## Import necessary dependencies

In [ ]:
import os
import yaml
from texar.torch import HParams

from forte.pipeline import Pipeline
from forte.data.ontology import conll03_ontology as conll
from forte.data.ontology.conll03_ontology import Sentence
from forte.data.readers.conll03_reader import CoNLL03Reader
from forte.processors.ner_predictor import (
    CoNLLNERPredictor,
)
from forte.common.resources import Resources


## Load the config 

In [ ]:
config_data = yaml.safe_load(open("config_data.yml", "r"))
config_model = yaml.safe_load(open("config_model.yml", "r"))

config = HParams({}, default_hparams=None)
config.add_hparam('config_data', config_data)
config.add_hparam('config_model', config_model)


## Load the resource

In [ ]:
resources: Resources = Resources()
resources.load(path=os.path.join(config.config_model.resource_dir,
                                 "resources.pkl"))


## Create the pipeline with CoNLL Reader and Predictor

In [ ]:
pl = Pipeline(resources)
pl.set_reader(CoNLL03Reader())
pl.add_processor(CoNLLNERPredictor(), config=config)

## Initialize the pipeline

In [ ]:
pl.initialize()

## Run the pipeline on the test dataset 

In [ ]:
for pack in pl.process_dataset(config.config_data.test_path):
    for pred_sentence in pack.get_data(
            context_type=Sentence,
            request={
                conll.Token:
                         {"fields": ["ner_tag"]},
                conll.Sentence: [],  # span by default
                conll.EntityMention: {}
            }):
        print("============================")
        print(pred_sentence["context"])
        print(pred_sentence["Token"]["ner_tag"])
        print("============================")
